Define the access key id and secret code for AWS.

In [34]:
access_key_id = ''
secret_access_key = ''

In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3

In [3]:
s3 = boto3.resource('s3', aws_access_key_id=access_key_id,
                    aws_secret_access_key=secret_access_key)
try:
    s3.create_bucket(Bucket='datacont-roman',
                     CreateBucketConfiguration={'LocationConstraint': 'us-west-2'})
except:
    print("this may already exist")

this may already exist


In [4]:
bucket = s3.Bucket("datacont-roman")
bucket.Acl().put(ACL='public-read')

{&#39;ResponseMetadata&#39;: {&#39;RequestId&#39;: &#39;EM0GAJAV1SANCX7R&#39;,
  &#39;HostId&#39;: &#39;iVA5Mdhd5Ik2D/xtouw017NBAsilT+HPqZShk9zzDyk3BT3aj5gNZr7iKMxSkRFKgAJtHxFltVA=&#39;,
  &#39;HTTPStatusCode&#39;: 200,
  &#39;HTTPHeaders&#39;: {&#39;x-amz-id-2&#39;: &#39;iVA5Mdhd5Ik2D/xtouw017NBAsilT+HPqZShk9zzDyk3BT3aj5gNZr7iKMxSkRFKgAJtHxFltVA=&#39;,
   &#39;x-amz-request-id&#39;: &#39;EM0GAJAV1SANCX7R&#39;,
   &#39;date&#39;: &#39;Mon, 05 Oct 2020 01:11:51 GMT&#39;,
   &#39;content-length&#39;: &#39;0&#39;,
   &#39;server&#39;: &#39;AmazonS3&#39;},
  &#39;RetryAttempts&#39;: 1}}

In [6]:
#upload a new object into the bucket
body = open('./exp1', 'rb')
o = s3.Object('datacont-roman', 'test').put(Body=body)
s3.Object('datacont-roman', 'test').Acl().put(ACL='public-read')

{&#39;ResponseMetadata&#39;: {&#39;RequestId&#39;: &#39;DD6012C6371F00BE&#39;,
  &#39;HostId&#39;: &#39;wZ3bruXybwY9dIeuLlpeMAJefbDWsxziGFCysvSMj3QfFP0E5OkL2TKAgRgdfCHDWEhngGXtqls=&#39;,
  &#39;HTTPStatusCode&#39;: 200,
  &#39;HTTPHeaders&#39;: {&#39;x-amz-id-2&#39;: &#39;wZ3bruXybwY9dIeuLlpeMAJefbDWsxziGFCysvSMj3QfFP0E5OkL2TKAgRgdfCHDWEhngGXtqls=&#39;,
   &#39;x-amz-request-id&#39;: &#39;DD6012C6371F00BE&#39;,
   &#39;date&#39;: &#39;Mon, 05 Oct 2020 01:12:16 GMT&#39;,
   &#39;content-length&#39;: &#39;0&#39;,
   &#39;server&#39;: &#39;AmazonS3&#39;},
  &#39;RetryAttempts&#39;: 0}}

In [8]:
dyndb = boto3.resource('dynamodb', region_name='us-west-2',
                       aws_access_key_id=access_key_id, aws_secret_access_key=secret_access_key)
try:
    table = dyndb.create_table(
        TableName='DataTable',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except:
    # if there is an exception, the table may already exist. if so...
    table = dyndb.Table("DataTable")

# wait for the table to be created
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

print(table.item_count)

0


In [17]:
import csv

Reading the csv file, uploading the blobs and creating the table

In [31]:
with open('./experiments.csv', 'rt') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        print(item)
        body = open('./'+item[3], 'rb')
        s3.Object('datacont-roman', item[3]).put(Body=body)
        md = s3.Object('datacont-roman', item[3]).Acl().put(ACL='public-read')

        url = " https://s3-us-west-2.amazonaws.com/datacont-roman/"+item[3]
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
                         'description': item[4], 'date': item[2], 'url': url}
        try:
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

[&#39;experiment1&#39;, &#39;1&#39;, &#39;3/15/2002&#39;, &#39;exp1&#39;, &#39;this is the comment&#39;]
[&#39;experiment1&#39;, &#39;2&#39;, &#39;3/15/2002&#39;, &#39;exp2&#39;, &#39;this is the comment2&#39;]
[&#39;experiment2&#39;, &#39;3&#39;, &#39;3/16/2002&#39;, &#39;exp3&#39;, &#39;this is the comment3&#39;]
[&#39;experiment3&#39;, &#39;4&#39;, &#39;3/16/2002&#39;, &#39;exp4&#39;, &#39;this is the comment233&#39;]


In [32]:
response = table.get_item(
    Key={
        'PartitionKey': 'experiment3',
        'RowKey': '4'
    }
)
item = response['Item']
print(item)

{&#39;PartitionKey&#39;: &#39;experiment3&#39;, &#39;RowKey&#39;: &#39;4&#39;, &#39;date&#39;: &#39;3/16/2002&#39;, &#39;description&#39;: &#39;this is the comment233&#39;, &#39;url&#39;: &#39; https://s3-us-west-2.amazonaws.com/datacont-roman/exp4&#39;}


In [33]:
response

{&#39;Item&#39;: {&#39;PartitionKey&#39;: &#39;experiment3&#39;,
  &#39;RowKey&#39;: &#39;4&#39;,
  &#39;date&#39;: &#39;3/16/2002&#39;,
  &#39;description&#39;: &#39;this is the comment233&#39;,
  &#39;url&#39;: &#39; https://s3-us-west-2.amazonaws.com/datacont-roman/exp4&#39;},
 &#39;ResponseMetadata&#39;: {&#39;RequestId&#39;: &#39;7NNA8L1R02U1DP6FL8RSIQQCS7VV4KQNSO5AEMVJF66Q9ASUAAJG&#39;,
  &#39;HTTPStatusCode&#39;: 200,
  &#39;HTTPHeaders&#39;: {&#39;server&#39;: &#39;Server&#39;,
   &#39;date&#39;: &#39;Mon, 05 Oct 2020 01:24:05 GMT&#39;,
   &#39;content-type&#39;: &#39;application/x-amz-json-1.0&#39;,
   &#39;content-length&#39;: &#39;204&#39;,
   &#39;connection&#39;: &#39;keep-alive&#39;,
   &#39;x-amzn-requestid&#39;: &#39;7NNA8L1R02U1DP6FL8RSIQQCS7VV4KQNSO5AEMVJF66Q9ASUAAJG&#39;,
   &#39;x-amz-crc32&#39;: &#39;1765371357&#39;},
  &#39;RetryAttempts&#39;: 0}}